In [7]:
import SimpleITK as sitk  # Imports SimpleITK (sitk) for reading and processing medical images.
import numpy as np        # Imports NumPy (np) for numerical operations, array manipulation, and creating the dummy image.

# --- Configuration ---
# You need to download your COVID-19 Chest CT data from TCIA.
# If using DICOM files, set this to the directory containing the DICOM series.
# If using a single file (like .nii or .mha), set this to the file path.
DATA_PATH = "path/to/your/TCIA_COVID_CT_directory_or_file.mha" # Path to the input data (MUST BE MODIFIED by the user).
OUTPUT_DIR = "output_masks/"                                   # Directory where output segmentation masks will be saved.

# Create the output directory if it doesn't exist
import os                                                      # Imports the operating system interface module.
os.makedirs(OUTPUT_DIR, exist_ok=True)                         # Creates the output directory; exist_ok=True avoids error if it exists.

def load_3d_image(path):
    """Loads a 3D image, handling both DICOM series and single files."""
    try:
        if os.path.isdir(path):                                # Checks if the provided path is a directory (suggesting a DICOM series).
            # Read DICOM series
            reader = sitk.ImageSeriesReader()                  # Creates a SimpleITK object to read a series of images.
            dicom_names = reader.GetFileNamesForSeries(path)   # Gets the ordered list of file paths belonging to the DICOM series.
            reader.SetFileNames(dicom_names)                   # Tells the reader which files to process.
            image = reader.Execute()                           # Reads the DICOM files and creates the 3D volume image.
        else:
            # Read a single file (MHA, NII, etc.)
            image = sitk.ReadImage(path)                       # Uses the general reader for single volume files (MHA, NII, etc.).

        # Cast to float32 for processing (standard practice for ITK filters)
        image = sitk.Cast(image, sitk.sitkFloat32)             # Converts the image pixel data type to 32-bit float for reliable filtering.
        print(f"✅ Image loaded successfully. Size: {image.GetSize()}") # Prints success message and the dimensions of the loaded image.
        return image                                           # Returns the loaded SimpleITK image object.

    except Exception as e:                                     # Catches any error that occurs during the loading process.
        print(f"❌ Error loading image: {e}")                   # Prints the specific error message.
        # Placeholder image for execution if real data is missing
        print("   Using a dummy image for demonstration.")      # Informs the user a fallback is being used.
        # Creates a 64x64x64 array of zeros, converts it to a SITK image, and casts it to float32.
        dummy_array = np.zeros((64, 64, 64), dtype=np.int16)
        dummy_sitk = sitk.GetImageFromArray(dummy_array)
        return sitk.Cast(dummy_sitk, sitk.sitkFloat32)

# Load the image
ct_image = load_3d_image(DATA_PATH)                            # Calls the function to load the CT data from the defined path.

❌ Error loading image: Exception thrown in SimpleITK ImageFileReader_Execute: /Users/ec2-user/actions-runner/_work/SimpleITK/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:91:
sitk::ERROR: The file "path/to/your/TCIA_COVID_CT_directory_or_file.mha" does not exist.
   Using a dummy image for demonstration.


In [3]:
def segment_lesion_b3(image):
    """
    Implements B3: Region Growing Segmentation for a COVID-19 lesion.
    NOTE: Requires manual input of a seed point within the lesion.
    """
    print("\n--- Running B3: Region Growing for a Lesion ---")
    
    # --- MANUAL INPUT REQUIRED ---
    # You MUST inspect your data (e.g., in ImageJ) and find the (x, y, z) 
    # physical coordinates of a point inside a lesion to use as a seed.
    SEED_PHYSICAL_POINT = (32.0, 32.0, 32.0) # Placeholder: Replace with actual (x,y,z)

    # Define the acceptable range of intensity for the lesion to grow into.
    # Lesions (GGO/Consolidation) are typically between -300 HU and +100 HU.
    LOWER_BOUND = -300.0 
    UPPER_BOUND = 100.0

    # Get the image index corresponding to the physical point
    try:
        seed_index = image.TransformPhysicalPointToIndex(SEED_PHYSICAL_POINT)
    except Exception as e:
        print(f"❌ Could not transform physical point to index. Using image center (0,0,0) as seed.")
        seed_index = image.GetSize() // 2 # Center index of placeholder
        
    print(f"   1. Using seed point index: {seed_index}")
    print(f"   2. Growing region within HU range: [{LOWER_BOUND}, {UPPER_BOUND}]")

    # SimpleITK's ConnectedThresholdImageFilter implements the core idea of Region Growing.
    lesion_mask = sitk.ConnectedThreshold(
        image,
        seedList=[seed_index],
        lower=LOWER_BOUND,
        upper=UPPER_BOUND,
        replaceValue=1 # Value for the grown lesion region
    )
    
    # Optional refinement: Post-processing to clean up the mask (Mathematical Morphology)
    # The project suggests using morphological operators [cite: 107, 119]
    # Here, we use a closing filter to fill small holes and smooth boundaries.
    closed_mask = sitk.BinaryMorphologicalClosing(
        lesion_mask, 
        kernelRadius=[3, 3, 3], # Adjust radius based on lesion size
        foregroundValue=1
    )
    print("   3. Region Growing and Morphological Closing applied.")

    # Save the result
    output_path = os.path.join(OUTPUT_DIR, "b3_lesion_segmentation.mha")
    sitk.WriteImage(sitk.Cast(closed_mask, sitk.sitkUInt8), output_path)
    print(f"   Mask saved to: {output_path}")
    
    return closed_mask

# Execute B3 Segmentation
lesion_mask = segment_lesion_b3(ct_image)


--- Running B3: Region Growing for a Lesion ---
   1. Using seed point index: (32, 32, 32)
   2. Growing region within HU range: [-300.0, 100.0]
   3. Region Growing and Morphological Closing applied.
   Mask saved to: output_masks/b3_lesion_segmentation.mha


In [6]:
# First, define the lung_mask and lesion_mask variables
# For example, you might load them from files using SimpleITK
import SimpleITK as sitk

# Load your mask files
# Replace these with your actual file paths where the mask files exist
lung_mask_path = "/path/to/your/actual/lung_mask.nii.gz"  # Update this path
lesion_mask_path = "/path/to/your/actual/lesion_mask.nii.gz"  # Update this path

# Check if files exist before loading
import os

if os.path.exists(lung_mask_path) and os.path.exists(lesion_mask_path):
    lung_mask = sitk.ReadImage(lung_mask_path)
    lesion_mask = sitk.ReadImage(lesion_mask_path)
    
    def perform_quantification(mask, mask_name):
        """
        Performs basic volume and shape quantification on a segmented mask.
        """
        print(f"\n--- Quantification for: {mask_name} ---")
        
        # Use LabelShapeStatisticsImageFilter for physical measurements
        label_stats = sitk.LabelShapeStatisticsImageFilter()
        # Execute only on the part of the image that is labeled '1'
        label_stats.Execute(sitk.Cast(mask, sitk.sitkInt16)) 

        if label_stats.HasLabel(1):
            # 1. Volume Quantification (Task e, g)
            volume_mm3 = label_stats.GetPhysicalSize(1)
            
            # 2. Shape Quantification (Task e)
            # Bounding box is the smallest rectangle that encloses the object.
            bounding_box = label_stats.GetBoundingBox(1) 
            
            # Elongation measures how stretched or non-spherical the object is.
            elongation = label_stats.GetElongation(1)
            
            print(f"   Total Volume (mm³): {volume_mm3:.2f}")
            print(f"   Object Elongation: {elongation:.2f}")
            print(f"   Bounding Box (min/max x, y, z): {bounding_box}")
        else:
            print("   No label '1' found in the mask to quantify.")

    # Perform quantification on the results
    perform_quantification(lung_mask, "Lung Field (B1)")
    perform_quantification(lesion_mask, "COVID-19 Lesion (B3)")
else:
    if not os.path.exists(lung_mask_path):
        print(f"Error: The lung mask file '{lung_mask_path}' does not exist.")
    if not os.path.exists(lesion_mask_path):
        print(f"Error: The lesion mask file '{lesion_mask_path}' does not exist.")

Error: The lung mask file '/path/to/your/actual/lung_mask.nii.gz' does not exist.
Error: The lesion mask file '/path/to/your/actual/lesion_mask.nii.gz' does not exist.
